# XGBoost `XGBRegressor` — Hiperparâmetros no **construtor** vs **treino** (fit)

> Autor: Lucas Bussinger — Gerado pelo M365 Copilot em 2026-01-27 12:14
> Objetivo: Demonstrar, com exemplos executáveis, **onde** cada hiperparâmetro do `XGBRegressor` é definido: no **modelo** (construtor) ou no **treino** (`fit`).


## Sumário
1. [Visão geral rápida](#visao-geral)
2. [Parâmetros no **construtor** (`XGBRegressor(...)`)](#construtor)
3. [Parâmetros no **treino** (`fit(...)`)](#treino)
4. [Exemplo completo com validação e *early stopping*](#exemplo-completo)
5. [Presets: GBTREE, DART e GBLINEAR](#presets)
6. [Inspecionando todos os parâmetros do estimador](#inspecao)


## 1) Visão geral rápida <a id="visao-geral"></a>
- **Construtor**: define *hiperparâmetros do modelo/booster* (e.g., `n_estimators`, `learning_rate`, `max_depth`, `subsample`, `colsample_*`, `reg_alpha`, `reg_lambda`, `tree_method`, `gamma`, `booster`, `objective`, etc.).
- **Treino (`fit`)**: define *parâmetros de avaliação e rotina de treinamento* (e.g., `eval_set`, `eval_metric`, `early_stopping_rounds`, `verbose`, `callbacks`, `sample_weight`, `sample_weight_eval_set`).


## 2) Parâmetros no **construtor** (`XGBRegressor(...)`) <a id="construtor"></a>
São passados na criação do estimador e controlam o **booster** e a **regularização**.


In [ ]:
gbtree = XGBRegressor(
    # --- GERAL ---
    booster="gbtree",          # [gbtree, dart] | Algoritmo base. gbtree é o padrão.
    n_estimators=300,          # [100-1000] | Núm. de árvores. Alto: Overfitting/Lento. Baixo: Underfitting.
    learning_rate=0.05,        # [0.01-0.3] | Passo de aprendizado. Alto: Rápido/Instável. Baixo: Robusto/Lento.
    random_state=42,           # [Any Int] | Semente para replicabilidade.

    # --- COMPLEXIDADE / ESTRUTURA (Crucial contra Overfitting) ---
    max_depth=4,               # [3-10] | Profundidade da árvore. Alto: Overfitting (seu 8 era alto). Baixo: Underfitting.
    min_child_weight=15.0,     # [1-20+] | Peso mín. na folha. Alto: Conservador (Geralmente melhor). Baixo: Overfitting.
    gamma=1.0,                 # [0-10+] | Ganho mín. para split (Poda). Alto: Árvores menores. Baixo: Complexidade.
    max_delta_step=0.0,        # [0-10] | Útil em classes desbalanceadas (raro em regressão).
    max_leaves=0,              # [0-255] | Máx. de folhas. 0 usa max_depth. Alto: Complexo.
    grow_policy="depthwise",   # [depthwise, lossguide] | Como a árvore cresce.
    max_bin=256,               # [256] | Discretização de variáveis. Alto: Mais precisão/Lento.

    # --- AMOSTRAGEM (Aleatoriedade para Generalizar) ---
    subsample=0.6,             # [0.5-1.0] | % de linhas por árvore. Alto: Overfitting. Baixo: Robusto.
    colsample_bytree=0.4,      # [0.3-0.8] | % de colunas por árvore. Alto: Overfitting. Baixo: Generalização (Força outras bandas).
    colsample_bylevel=1.0,     # [0.5-1.0] | % de colunas por nível.
    colsample_bynode=1.0,      # [0.5-1.0] | % de colunas por nó.

    # --- REGULARIZAÇÃO (Penalidade para o modelo) ---
    reg_lambda=20.0,           # [1-100] | L2 (Ridge). Alto: Modelo simples/estável. Baixo: Complexidade/Overfitting.
    reg_alpha=10.0,            # [0-100] | L1 (Lasso). Alto: Seleção de variáveis (ignora ruído). Baixo: Usa tudo.

    # --- MÉTODO DE CONSTRUÇÃO ---
    tree_method="hist",        # [auto, exact, approx, hist] | 'hist' é o mais rápido para datasets médios/grandes.
    sampling_method="uniform", # [uniform, gradient_based] | Método de sorteio de amostras.

    # --- DIVERSOS ---
    base_score=0.5,            # Ponto de partida das predições (geralmente a média do alvo).
    verbosity=1,               # [0,1,2,3] | Nível de detalhe dos logs de treino.
    importance_type="gain",    # [gain, weight, cover] | 'gain' é o melhor para entender o sinal real das features.

    # --- OBJETIVO ---
    objective="reg:squarederror", # Erro quadrático médio para regressão.
    enable_categorical=False      # Se True, trata colunas 'category' automaticamente.
)

# DICA: Para o seu Challenge, os valores que alterei acima (max_depth menor, 
# reg_alpha/lambda maiores e colsample menor) são focados em fechar aquele 
# "gap" na curva de aprendizado e melhorar sua nota na submissão.

### Booster DART (parâmetros adicionais)
Além dos parâmetros de árvores, o DART adiciona *dropout* de árvores.


In [ ]:
dart = XGBRegressor(
    # --- GERAL (DART é mais lento, pode precisar de menos LR ou mais Estimators) ---
    booster="dart",            # Ativa o modo Dropout. Excelente contra Overfitting.
    n_estimators=400,          # [100-1000] | Núm. de árvores. DART converge mais devagar que gbtree.
    learning_rate=0.05,        # [0.01-0.2] | Passo. No DART, valores menores são mais estáveis.
    
    # --- ESTRUTURA DA ÁRVORE (Mesmos do gbtree) ---
    max_depth=4,               # [3-6] | Profundidade. Alto: Overfitting. Baixo: Generalização. (Reduzi de 6 para 4).
    min_child_weight=10.0,     # [1-20+] | Mín. de dados por folha. Alto: Bloqueia decoreba. Baixo: Overfitting.
    gamma=0.5,                 # [0-5] | Penalidade para novos splits. Alto: Conservador.
    subsample=0.6,             # [0.5-0.9] | % de linhas por árvore. Baixo: Mais aleatoriedade/Robusto.
    colsample_bytree=0.5,      # [0.3-0.7] | % de colunas por árvore. Baixo: Força o uso de bandas fracas.

    # --- ESPECÍFICOS DO DART (A mágica do Dropout) ---
    rate_drop=0.15,            # [0.0-0.3] | Fração de árvores a serem "dropadas" por rodada.
                               # Alto: Regularização forte, impede especialização. Baixo: Vira gbtree comum.
    
    skip_drop=0.5,             # [0.0-0.9] | Probabilidade de pular o dropout em uma iteração.
                               # Alto: Mais chance de agir como gbtree normal. Baixo: Dropout agressivo.
    
    one_drop=False,            # [True, False] | Se True, pelo menos uma árvore será dropada sempre.
    
    max_drop=50,               # [0-n] | Máximo de árvores que podem ser dropadas por rodada. 0 = sem limite.
    
    sample_type="uniform",     # [uniform, weighted] | Como escolher as árvores p/ dropar. 
                               # 'uniform': todas iguais. 'weighted': dropa conforme o peso.
    
    normalize_type="tree",     # [tree, forest] | Como reescalar as árvores após o dropout.
                               # 'tree': Nova árvore tem peso 1/k. 'forest': Mais complexo (estilo floresta).

    # --- REGULARIZAÇÃO EXTRA ---
    reg_lambda=15.0,           # [1-100] | L2. Alto: Reduz oscilações nas predições.
    reg_alpha=10.0,            # [0-100] | L1. Alto: Ignora colunas que são apenas ruído.

    # --- DIVERSOS ---
    tree_method="hist",        # Método rápido de construção.
    importance_type="gain",    # Mede a importância baseada no ganho real de informação.
    objective="reg:squarederror"
)

# NOTA PARA O CHALLENGE:
# O seu código original tinha max_depth=6 e subsample/colsample=0.9. 
# Isso estava dando 90% de liberdade para o modelo decorar o mapa.
# Com as reduções que fiz acima, o DART será forçado a criar padrões 
# que funcionam nos dados de teste que ele nunca viu.

### Booster GBLINEAR (modelo linear)
Ignora hiperparâmetros de árvore e usa principalmente regularização.


In [ ]:

blinear = XGBRegressor(
    booster="gblinear",
    n_estimators=200,      # iterações do otimizador
    learning_rate=0.1,     # também usado no otimizador linear

    # Regularização
    reg_lambda=1.0,        # L2
    reg_alpha=0.0,         # L1

    # Algumas versões/compilações expõem:
    # lambda_bias=0.0,     # L2 no viés (se disponível)
    # feature_selector="cyclic",  # "cyclic" | "thrifty" | "greedy" (contexto interno)
    # top_k=0              # int >= 0
)


## 3) Parâmetros no **treino** (`fit(...)`) <a id="treino"></a>
Usados para **avaliação**, **validação**, *early stopping* e **pesos de amostra**.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import numpy as np

# Dados sintéticos para exemplo
X, y = make_regression(n_samples=5000, n_features=30, noise=15.0, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = gbtree

sample_weight = np.ones(len(X_train), dtype=float)
sample_weight_val = np.ones(len(X_val), dtype=float)

model.fit(
    X_train, y_train,
    # Avaliação/validação
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric=["rmse", "mae"],  # métrica de AVALIAÇÃO

    # Early stopping (requer eval_set)
    early_stopping_rounds=50,
    verbose=True,

    # Pesos
    sample_weight=sample_weight,
    sample_weight_eval_set=[sample_weight, sample_weight_val],
)

best_iter = getattr(model, 'best_iteration', None)
best_score = getattr(model, 'best_score', None)
best_iter, best_score


## 3.2  parametros no treino(fit)

In [ ]:

from sklearn.model_selection import train_test_split
import numpy as np

# Suponha X, y já preparados (numpy arrays ou pandas)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = gbtree  # de cima

# Exemplo de pesos por amostra (opcional)
sample_weight = np.ones(len(X_train), dtype=float)
sample_weight_val = np.ones(len(X_val), dtype=float)

model.fit(
    X_train, y_train,

    # ---------------------------
    # Avaliação/validação
    # ---------------------------
    eval_set=[(X_train, y_train), (X_val, y_val)],  # lista de tuplas
    eval_metric=[                                   # string ou lista de strings
        "rmse",     # comuns em regressão: "rmse", "mae", "mape"
        # Para objetivos específicos:
        # "poisson-nloglik", "gamma-nloglik", "aft-nloglik", etc.
    ],

    # ---------------------------
    # Early stopping (para parar nos melhores rounds)
    # ---------------------------
    early_stopping_rounds=50,   # int >= 1 (requer eval_set)
    verbose=True,               # imprime métricas a cada iteração

    # ---------------------------
    # Pesos e amostragem por instância
    # ---------------------------
    sample_weight=sample_weight,                 # (opcional)
    sample_weight_eval_set=[sample_weight, sample_weight_val],  # (opcional)

    # ---------------------------
    # Callbacks (avançado)
    # ---------------------------
    callbacks=None,             # pode passar lista de callbacks xgboost.callback.*
    # xgb_model=None,          # caminho/objeto para continuar treinamento (warm start)
)


## 4) Exemplo completo com validação e *early stopping* <a id="exemplo-completo"></a>


In [ ]:
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(n_samples=3000, n_features=25, noise=10.0, random_state=0)
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=0)

model_full = XGBRegressor(
    booster="gbtree",
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=1.0,
    subsample=0.85,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    gamma=0.0,
    tree_method="hist",
    objective="reg:squarederror",
    random_state=42,
    verbosity=1
)

model_full.fit(
    X_tr, y_tr,
    eval_set=[(X_tr, y_tr), (X_va, y_va)],
    eval_metric=["rmse", "mae"],
    early_stopping_rounds=100,
    verbose=False
)
print("Melhor iteração:", getattr(model_full, 'best_iteration', None))
print("Melhor pontuação (última métrica):", getattr(model_full, 'best_score', None))


## 5) Presets: GBTREE, DART e GBLINEAR <a id="presets"></a>
Blocos prontos para reutilização em diferentes cenários.


In [ ]:
def make_gbtree(random_state=42):
    return XGBRegressor(
        booster="gbtree", n_estimators=1000, learning_rate=0.03, max_depth=7,
        subsample=0.85, colsample_bytree=0.9, reg_lambda=1.0, reg_alpha=0.0,
        tree_method="hist", objective="reg:squarederror", random_state=random_state
    )

def make_dart(random_state=42):
    return XGBRegressor(
        booster="dart", n_estimators=800, learning_rate=0.05, max_depth=6,
        subsample=0.9, colsample_bytree=0.9, rate_drop=0.1, skip_drop=0.0,
        sample_type="uniform", normalize_type="tree", objective="reg:squarederror",
        tree_method="hist", random_state=random_state
    )

def make_gblinear(random_state=42):
    return XGBRegressor(
        booster="gblinear", n_estimators=400, learning_rate=0.1,
        reg_lambda=1.0, reg_alpha=0.0, objective="reg:squarederror",
        random_state=random_state
    )

gbtree_preset = make_gbtree()
dart_preset = make_dart()
gblinear_preset = make_gblinear()
gbtree_preset, dart_preset, gblinear_preset


## 6) Inspecionando todos os parâmetros do estimador <a id="inspecao"></a>
Útil para descobrir tudo que o wrapper sklearn expõe na sua instalação.


In [ ]:
# A saída pode variar conforme a versão do XGBoost instalada.
XGBRegressor().get_params().keys()


---
### Notas
- `eval_metric` é parâmetro de **treino** (como avaliar), enquanto `objective` é do **modelo** (qual perda otimizar).
- `early_stopping_rounds` só funciona quando `eval_set` é fornecido.
- `n_estimators` define o **máximo** de boosting rounds; com *early stopping*, o número efetivo pode ser menor (`best_iteration`).
- Métricas comuns na regressão: `rmse`, `mae`, `mape`; para contagem: `poisson-nloglik`; para quantis (quando suportado): `quantile`.
- `tree_method="gpu_hist"` requer XGBoost com suporte a GPU.
